In [8]:
import lib as l

In [9]:
embeddings_gasoline = l.EMB.Embeddings("gasoline", ["sp", "eng", "ger"])

In [10]:
t, tr, algo = l.EMB.get_best_thresholds(embeddings_gasoline)

Multiprocessing done, took 6.625 minutes
Best parameters : t = 6.5 ; tr = 6.75 ; algo = louvain


In [11]:
l.EMB.save_graph_communities(t,tr, embeddings_gasoline)

In [12]:
l.output_notebook()

Loading BokehJS ...

In [13]:
algo="louvain"
df = l.pd.read_csv('data/graphs/graph_gasoline.csv')
G = l.nx.from_pandas_edgelist(df, 'from', 'to')
communities = l.community.louvain_communities(G) if algo == "louvain" else l.community.greedy_modularity_communities(G)     

In [14]:
title = 'Topics Network Gasoline'
df = l.pd.read_csv('data/graphs/graph_gasoline.csv')
G = l.nx.from_pandas_edgelist(df, 'from', 'to')
#degrees = dict(nx.degree(G))
#nx.set_node_attributes(G, name='degree', values=degrees)
#number_to_adjust_by = 2
#adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
#nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
communities = l.community.louvain_communities(G)
# Create empty dictionaries
modularity_class = {}
modularity_color = {}

#Loop through each community in the network
for community_number, comm in enumerate(communities):
    #For each member of the community, add their community number and a distinct color
    for name in comm: 
        modularity_class[name] = community_number
        modularity_color[name] =l.Bokeh8[(community_number*3+1)%8]

# Add modularity class and color as attributes from the network above
l.nx.set_node_attributes(G, modularity_class, 'Cluster')
l.nx.set_node_attributes(G, modularity_color, 'Color')

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
#size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'Color'


#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Topic", "@index"),
         ("Cluster", "@Cluster"),
]

#Create a plot — set dimensions, toolbar, and title
plot = l.figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset, tap", active_scroll='wheel_zoom',
            x_range=l.Range1d(-10.1, 10.1), y_range=l.Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = l.from_networkx(G, l.nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = l.Circle(size=8, fill_color=color_by_this_attribute)

#Set edge opacity and width
network_graph.edge_renderer.glyph = l.MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

l.show(plot)
#save(plot, filename=f"{title}.html")